<a href="https://colab.research.google.com/github/Slickkiddo/DL-professional-Certificate/blob/main/NLP_training_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
print(tf.__version__)

# !pip install -q tensorflow-datasets

2.3.0


In [5]:
import tensorflow_datasets as tfds

imdb, info = tfds.load("imdb_reviews", with_info = True, as_supervised = True)

In [8]:
import numpy as np

train_data, test_data = imdb['train'], imdb['test']

training_sentences, training_labels = [], []
testing_sentences, testing_labels = [], []

for s,l in train_data:
  training_sentences+=[s.numpy().decode('utf8')]
  training_labels+=[l.numpy()]

for s,l in test_data:
  testing_sentences+=[s.numpy().decode('utf8')]
  testing_labels+=[l.numpy()]

print(training_sentences[0])
print(training_labels[0])

This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.
0


In [10]:
vocab_size = 10000
embedding_dim = 16
oov_tok = "<OOV>"
trunc_type = "post"
max_length = 120

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words = vocab_size, oov_token = oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen = max_length, truncating = trunc_type)

testing_sequences = tokenizer.texts_to_sequences(testing_sentences)
test_padded = pad_sequences(testing_sequences, maxlen=max_length)

In [24]:
reverse_word_index = dict([(val,key) for key,val in word_index.items()])

def decode_review(text):
  return ' '.join([reverse_word_index.get(i,'?') for i in text])

print(decode_review(padded[0]))
print(training_sentences[0])

? ? ? this was an absolutely terrible movie don't be <OOV> in by christopher walken or michael <OOV> both are great actors but this must simply be their worst role in history even their great acting could not redeem this movie's ridiculous storyline this movie is an early nineties us propaganda piece the most pathetic scenes were those when the <OOV> rebels were making their cases for <OOV> maria <OOV> <OOV> appeared phony and her pseudo love affair with walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning i am disappointed that there are movies like this ruining actor's like christopher <OOV> good name i could barely sit through it
This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pa

In [25]:
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length = max_length),
                              tf.keras.layers.Flatten(),
                              tf.keras.layers.Dense(6, activation = "relu"),
                             tf.keras.layers.Dense(1, activation = "sigmoid")])

model.compile(loss = "binary_crossentropy", optimizer = "adam", metrics = ['accuracy'])
model.summary()



Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 120, 16)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 1920)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 6)                 11526     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 7         
Total params: 171,533
Trainable params: 171,533
Non-trainable params: 0
_________________________________________________________________


In [26]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)

n_epochs = 10
model.fit(padded, training_labels_final, epochs = n_epochs, validation_data = (test_padded, testing_labels_final))

Epoch 1/10
782/782 [==============================] - 4s 5ms/step - loss: 0.4983 - accuracy: 0.7368 - val_loss: 0.3462 - val_accuracy: 0.8479
Epoch 2/10
782/782 [==============================] - 4s 5ms/step - loss: 0.2360 - accuracy: 0.9085 - val_loss: 0.3668 - val_accuracy: 0.8425
Epoch 3/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0851 - accuracy: 0.9792 - val_loss: 0.4549 - val_accuracy: 0.8294
Epoch 4/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0219 - accuracy: 0.9972 - val_loss: 0.5334 - val_accuracy: 0.8264
Epoch 5/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0069 - accuracy: 0.9992 - val_loss: 0.5917 - val_accuracy: 0.8250
Epoch 6/10
782/782 [==============================] - 4s 5ms/step - loss: 0.0022 - accuracy: 0.9999 - val_loss: 0.6451 - val_accuracy: 0.8251
Epoch 7/10
782/782 [==============================] - 4s 5ms/step - loss: 9.3834e-04 - accuracy: 1.0000 - val_loss: 0.6862 - val_accuracy: 0.8276
Ep

In [27]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape) # shape: (vocab_size, embedding_dim)

(10000, 16)


In [28]:
import io

out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
out_m = io.open('meta.tsv', 'w', encoding='utf-8')
for word_num in range(1, vocab_size):
  word = reverse_word_index[word_num]
  embeddings = weights[word_num]
  out_m.write(word + "\n")
  out_v.write('\t'.join([str(x) for x in embeddings]) + "\n")
out_v.close()
out_m.close()

In [31]:
try:
  from google.colab import files
except ImportError:
  pass
else:
  files.download('vecs.tsv')
  files.download('meta.tsv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [30]:
sentence = "I really think this is amazing. honest."
sequence = tokenizer.texts_to_sequences([sentence])
print(sequence)

[[11, 64, 102, 12, 7, 478, 1200]]
